In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [2]:
# Note: For the canopy_height.download_new_tiles to work, you need to create an AWS account, then create a file named .aws/credentials in your home directory (e.g. /home/147/cb8590) with these contents:
# [default]
# aws_access_key_id = ACCESS_KEY
# aws_secret_access_key = SECRET_KEY

In [3]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import gdata_dir, scratch_dir
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.silo_daily import silo_daily
from DAESIM_preprocess.canopy_height import identify_relevant_tiles, download_new_tiles, merge_tiles

In [4]:
# Choosing location
lat, lon = -34.3890427, 148.469499
buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total
stub = "Test"
start_year = 2020
end_year = 2021

# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = scratch_dir

In [5]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/Test_terrain_original.tif
Saved /g/data/xe2/cb8590/Data/PadSeg/Test_terrain.tif
CPU times: user 105 ms, sys: 25.4 ms, total: 130 ms
Wall time: 352 ms


In [ ]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable unfortunately
variables = ['Clay', 'Sand', 'Silt', 'pH_CaCl2']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Downloaded /g/data/xe2/cb8590/Data/PadSeg/Test_Clay.tif
Downloaded /g/data/xe2/cb8590/Data/PadSeg/Test_Sand.tif


In [ ]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

In [ ]:
%%time
# Download modelled variables from OzWald
variables = ['Ssoil']
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

In [ ]:
%%time
# Download daily weather data hosted on OzWald
variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

In [ ]:
%%time
ds_silo_daily = silo_daily(["daily_rain", "max_temp", "min_temp", "et_morton_actual", "et_morton_potential"], lat, lon, buffer, start_year, end_year, outdir, stub)
ds_silo_daily

In [ ]:
%%time 
tiles = identify_relevant_tiles(lat, lon, buffer)
download_new_tiles(tiles)
merge_tiles(lat, lon, buffer, outdir, stub, tmp_dir)